In [71]:
import json
from tqdm import tqdm

In [72]:
with open('conversations.json', 'r') as f:
    conversations = json.load(f)

In [83]:
def get_msg(convo):
    mapping = convo['mapping']
    user_msgs = []
    
    for node in mapping.keys():
        if mapping[node]['message']:
            if 'parts' in mapping[node]['message']['content'].keys() and mapping[node]['message']['author']['role'] == 'user':
                if isinstance(mapping[node]['message']['content']['parts'][0], str):
                    user_msgs.append(mapping[node]['message']['content']['parts'])
    return user_msgs

In [84]:
all_user_msgs = []
for convo in tqdm(conversations):
    all_user_msgs.extend(get_msg(convo))

len(all_user_msgs)

100%|██████████| 3261/3261 [00:00<00:00, 8199.67it/s]


14157

In [ ]:
from openai import OpenAI
import os 
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
embeddings = []

for t in tqdm(all_user_msgs):
    try:
        embedding = client.embeddings.create(input=t, model="text-embedding-3-small").data[0].embedding
        embeddings.append(embedding)
    except Exception as e:
        print(e)
        break




  2%|▏         | 214/14157 [00:56<1:01:42,  3.77it/s]


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 30309 tokens (30309 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}